In [198]:
import binascii

from cryptography import x509
from cryptography.hazmat.backends import default_backend

certificate_path = "/etc/ssl/certs/GlobalSign_ECC_Root_CA_-_R4.pem" #Certum_Trusted_Network_CA.pem"
cert_file = open(certificate_path, 'rb')
pem_data = cert_file.read()
cert = x509.load_pem_x509_certificate(pem_data, default_backend())
cert.extensions

<Extensions([<Extension(oid=<ObjectIdentifier(oid=2.5.29.15, name=keyUsage)>, critical=True, value=<KeyUsage(digital_signature=False, content_commitment=False, key_encipherment=False, data_encipherment=False, key_agreement=False, key_cert_sign=True, crl_sign=True, encipher_only=None, decipher_only=None)>)>, <Extension(oid=<ObjectIdentifier(oid=2.5.29.19, name=basicConstraints)>, critical=True, value=<BasicConstraints(ca=True, path_length=None)>)>, <Extension(oid=<ObjectIdentifier(oid=2.5.29.14, name=subjectKeyIdentifier)>, critical=False, value=<SubjectKeyIdentifier(digest=b'T\xb0{\xadE\xb8\xe2@\x7f\xfb\nn\xfb\xbe3\xc9<\xa3\x84\xd5')>)>])>

In [182]:
from nfc.ndef.m2m import m2m_certificate_format as m2m
from nfc.ndef.m2m.m2m_certificate_format import *

In [183]:
#  name = cert.issuer or cert.subject
def x509_name_to_m2m_name(x509_name):
    oid2m2m_map = {
               #'BUSINESS_CATEGORY', 
               'COMMON_NAME':'commonName', 
               'COUNTRY_NAME':'country', 
               'DN_QUALIFIER':'distinguishedNameQualifier', 
               'DOMAIN_COMPONENT':'domainComponent', 
               #'EMAIL_ADDRESS', 
               #'GENERATION_QUALIFIER', 
               #'GIVEN_NAME', 
               #'JURISDICTION_COUNTRY_NAME', 
               #'JURISDICTION_LOCALITY_NAME', 
               #'JURISDICTION_STATE_OR_PROVINCE_NAME', 
               'LOCALITY_NAME':'locality', 
               'ORGANIZATIONAL_UNIT_NAME':'organizationalUnit', 
               'ORGANIZATION_NAME':'organization', 
               #'PSEUDONYM', 
               'SERIAL_NUMBER':'serialNumber', 
               'STATE_OR_PROVINCE_NAME':'stateOrProvince', 
               #'SURNAME', 
               #'TITLE'
                }

    # Some fields of the M2M AttributeValue that do not fit with X.509 NameOIDs
    # 'registeredId'
    # 'octetsName'

    from cryptography.x509.oid import NameOID
    attrs = [attr for attr in dir(NameOID) if not attr.startswith('_')]  # Get all the possible attributes of a name
#     print(attrs)

    all_attrs = {attr:x509_name.get_attributes_for_oid(getattr(NameOID, attr)) for attr in attrs}  # Get all the attributes in that name, some may be empty
    filled_attrs = {k: v for k, v in all_attrs.items() if v}  # Filter out the empty attributes
    # print(filled_attrs)

    m2m_attribute_values = []

    for attr_name, filled_attr in filled_attrs.items():
        attr = filled_attr[0]
        m2m_attr_name = oid2m2m_map[attr_name]
        m2m_attr_val = AttributeValue(**{m2m_attr_name:attr.value})
        m2m_attribute_values += [m2m_attr_val]

    m2m_name = Name.new(*m2m_attribute_values)
    
    return m2m_name

m2m_issuer = x509_name_to_m2m_name(cert.issuer)
m2m_subject = x509_name_to_m2m_name(cert.subject)

In [184]:
import datetime
def x509_datetime_to_m2m_datetime(x509_datetime):
    epoch = datetime.datetime.utcfromtimestamp(0)
    seconds = (cert.not_valid_after - epoch).total_seconds()
    return int(seconds).to_bytes(4, byteorder='big')

def x509_validity_to_m2m_validity(x509_cert):
    seconds = (x509_cert.not_valid_after - x509_cert.not_valid_before).total_seconds() # seconds since validFrom
    validFrom = x509_datetime_to_m2m_datetime(x509_cert.not_valid_before)
    validDuration = int(seconds).to_bytes(4, byteorder='big')
    
    return validFrom, validDuration

In [185]:
from cryptography.hazmat.primitives.serialization import Encoding
from cryptography.hazmat.primitives.serialization import PublicFormat
m2m_public_key = cert.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo)
print(binascii.hexlify(m2m_public_key))

b'3059301306072a8648ce3d020106082a8648ce3d03010703420004b8c679d38f6c250e9f2e39191c03a4ae9ae539070916ca63b1b986f88a57c157ce42fa73a1f76542ff1ec100b26e730effc721e518a4aad9713fa8d4b9ce8c1d'


As pointed out by CloudFlare (https://blog.cloudflare.com/ecdsa-the-digital-signature-algorithm-of-a-better-internet/), not many Certificate Authorities use ECC. The combination with SHA256 is even more rare: I haven't found such a certificate preinstalled on a standard Ubuntu 16.04 box that uses ECC & SHA256. 

In [191]:
algo_mapping = {
    (ec.EllipticCurvePublicKey, hashes.SHA256, ec.SECP192R1):AlgorithmObjectIdentifiers.ecdsa_with_sha256_secp192r1,
    (ec.EllipticCurvePublicKey, hashes.SHA256, ec.SECP224R1):AlgorithmObjectIdentifiers.ecdsa_with_sha256_secp224r1,
    (ec.EllipticCurvePublicKey, hashes.SHA256, ec.SECT233K1):AlgorithmObjectIdentifiers.ecdsa_with_sha256_sect233k1,
    #(ec.EllipticCurvePublicKey, hashes.SHA256, ec.SECP223R1):AlgorithmObjectIdentifiers.ecdsa_with_sha256_sect233r1,
    ("algo", hashes.SHA256, ec.SECP192R1):                   AlgorithmObjectIdentifiers.ecqv_with_sha256_secp192r1,
    ("algo", hashes.SHA256, ec.SECP224R1):                   AlgorithmObjectIdentifiers.ecqv_with_sha256_secp224r1,
    ("algo", hashes.SHA256, ec.SECT233K1):                   AlgorithmObjectIdentifiers.ecqv_with_sha256_sect233k1,
    #("algo", hashes.SHA256, ec.SECP223R1):                   AlgorithmObjectIdentifiers.ecqv_with_sha256_sect233r1,
    (rsa.RSAPublicKey, hashes.SHA256, None):                 AlgorithmObjectIdentifiers.rsa_with_sha256,
    (ec.EllipticCurvePublicKey, hashes.SHA256, ec.SECP256R1):AlgorithmObjectIdentifiers.ecdsa_with_sha256_secp256r1,
    ("algo", hashes.SHA256, ec.SECP256R1):                   AlgorithmObjectIdentifiers.ecqv_with_sha256_secp256r1
    }

# See RFC 5480, section 2.1.1.1.  Named Curve 
ec_curve_oid_map = {
    ec.SECP192R1: "1.2.840.10045.3.1.1", #http://oid-info.com/get/1.2.840.10045.3.1.1
    ec.SECP224R1: "1.3.132.0.33", #http://oid-info.com/get/1.3.132.0.33
    ec.SECT233K1: "1.3.132.0.26", #http://oid-info.com/get/1.3.132.0.26
    ec.SECP256R1: "1.2.840.10045.3.1.7" #http://oid-info.com/get/1.2.840.10045.3.1.7
}
from pyasn1.type import univ
from pyasn1.codec.der import encoder as der_encoder

def x509_algo_to_m2m_cAAlgorithm(x509_cert):
    public_key = x509_cert.public_key()

    try:
        if isinstance(public_key, rsa.RSAPublicKey):
            return (algo_mapping[(rsa.RSAPublicKey, 
                            type(x509_cert.signature_hash_algorithm), 
                            None)], None)
        elif isinstance(public_key, dsa.DSAPublicKey): # Not supported by M2M though
            return (algo_mapping[(dsa.DSAPublicKey, 
                            type(x509_cert.signature_hash_algorithm), 
                            None)], None) 
        elif isinstance(public_key, ec.EllipticCurvePublicKey):
            return (algo_mapping[(ec.EllipticCurvePublicKey, 
                            type(x509_cert.signature_hash_algorithm), 
                            type(public_key.curve))], 
                    der_encoder.encode(univ.ObjectIdentifier(ec_curve_oid_map[type(public_key.curve)])))
    except KeyError as key_error:
        print("Certificate cannot be converted to M2M, an algorithm cannot be encoded".format(key_error))
        raise key_error
    
ca_algo = x509_algo_to_m2m_cAAlgorithm(cert)
print(ca_algo)

(<AlgorithmObjectIdentifiers.ecdsa_with_sha256_secp256r1: ObjectIdentifier('2.16.840.1.114513.1.9')>, b'\x06\x08*\x86H\xce=\x03\x01\x07')


In [195]:
validFrom, validDuration = x509_validity_to_m2m_validity(cert)

tbs = TBSCertificate.new(version=0,
                         serialNumber=int(cert.serial_number).to_bytes(20, byteorder='big'),
                         subject=m2m_subject,
                         cAAlgorithm=str(ca_algo[0].value),
                         cAAlgParams=ca_algo[1], # EC PARAMETERS/Name: elliptic curve: http://oid-info.com/get/1.2.840.10045.3.1.7
                         issuer=m2m_issuer,
                         validFrom=validFrom, # optional
                         validDuration=validDuration,  # seconds since validFrom, optional
                         pKAlgorithm=str(ca_algo[0].value),  # Same as cAAlgorithm
                         # pKAlgParams="1.2.3.4", #Optional
                         pubKey=m2m_public_key,
                         # authKeyId=authkey, optional, See https://tools.ietf.org/html/rfc5280#section-4.2.1.1 for explanation
                         #subjKeyId=int(1).to_bytes(1, byteorder='big'),  # Key ID of the subject (which number do we give the private key?)
                         #keyUsage=0b10100000.to_bytes(1, byteorder='big'), # digitalSignature & keyEncipherment bit set
                         # See https://tools.ietf.org/html/rfc5280#section-4.2.1.3
                         #certificatePolicy="2.5.29.32.0",  # Anypolicy: http://www.oid-info.com/get/2.5.29.32.0
                         )

In [196]:
der_encoder.encode(tbs)

b'0\x82\x01\x0f\x04\x14\x00\x00\x00*8\xa4\x1c\x96\n\x04\xdeB\xb2(\xa5\x0b\xe84\x98\x02\x06\t`\x86H\x01\x86\xfeQ\x01\t\x04\n\x06\x08*\x86H\xce=\x03\x01\x0705\x0c\nGlobalSign\x0c\x1bGlobalSign ECC Root CA - R4\x0c\nGlobalSign\x04\x04\x7f\xff\xff\xff\x04\x04/^r\x7f05\x0c\nGlobalSign\x0c\x1bGlobalSign ECC Root CA - R4\x0c\nGlobalSign\x06\t`\x86H\x01\x86\xfeQ\x01\t\x04[0Y0\x13\x06\x07*\x86H\xce=\x02\x01\x06\x08*\x86H\xce=\x03\x01\x07\x03B\x00\x04\xb8\xc6y\xd3\x8fl%\x0e\x9f.9\x19\x1c\x03\xa4\xae\x9a\xe59\x07\t\x16\xcac\xb1\xb9\x86\xf8\x8aW\xc1W\xceB\xfas\xa1\xf7eB\xff\x1e\xc1\x00\xb2ns\x0e\xff\xc7!\xe5\x18\xa4\xaa\xd9q?\xa8\xd4\xb9\xce\x8c\x1d'

In [193]:
# See https://cryptography.io/en/latest/hazmat/primitives/asymmetric/serialization/#cryptography.hazmat.primitives.serialization.PublicFormat
m2m_public_key_pkcs1 = cert.public_key().public_bytes(Encoding.DER, PublicFormat.PKCS1) # only has the public key
m2m_public_key_spki = cert.public_key().public_bytes(Encoding.DER, PublicFormat.SubjectPublicKeyInfo) # also has the algorithm identifier

print(binascii.hexlify(m2m_public_key_pkcs1))
print()
print(binascii.hexlify(m2m_public_key_spki))

difflen = len(m2m_public_key_spki) - len(m2m_public_key_pkcs1)
algo_identifier = m2m_public_key_pkcs1[:difflen]

print()

print(binascii.hexlify(algo_identifier))

ValueError: EC public keys do not support PKCS1 serialization